In [1]:
import pickle
import numpy as np
from keras.models import Sequential
import keras.layers
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
np.random.seed(0)
train, valid, test = pickle.load(open('dataset.pickle', 'rb'))
train = train.fillna(0)
test = test.fillna(0)

In [3]:
x_train = train[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_train = train.iloc[:,-8:]
x_test = test[['raw_acc','proc_gyro','raw_magnet','watch_acceleration','watch_heading', 'location', 'location_quick_features', 'audio_naive', 'audio_properties', 'discrete', 'lf_measurements']]
y_test = np.asarray(test.iloc[:,-8:])
y_test_decode = []
for i in range(y_test.shape[0]):
    y_test_decode.insert(len(y_test_decode),np.argmax(y_test[i,:]))

In [4]:
scaler = MinMaxScaler() #scale features between 0 and 1
x_train = scaler.fit_transform(x_train)
x_train = np.asarray(x_train).reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = scaler.fit_transform(x_test)
x_test = np.asarray(x_test).reshape(x_test.shape[0], 1, x_test.shape[1])

In [5]:
model = Sequential()
#model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2]), bias_regularizer=L1L2(l1=0.01, l2=0.01)))
model.add(LSTM(500, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.2, noise_shape=None, seed=None))
model.add(BatchNormalization())
model.add(Dense(100, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
history = model.fit([x_train], batch_size=20, y=y_train, verbose=1, validation_split=0.25, 
          shuffle=True, epochs=5)

Train on 227768 samples, validate on 75923 samples
Epoch 1/5
227768/227768 [==============================] - 209s 919us/step - loss: 0.9966 - acc: 0.6385 - val_loss: 1.3015 - val_acc: 0.5320
Epoch 2/5
227768/227768 [==============================] - 206s 903us/step - loss: 0.7801 - acc: 0.7195 - val_loss: 1.4471 - val_acc: 0.5097
Epoch 3/5
227768/227768 [==============================] - 206s 905us/step - loss: 0.6878 - acc: 0.7530 - val_loss: 1.4621 - val_acc: 0.5064
Epoch 4/5
227768/227768 [==============================] - 207s 910us/step - loss: 0.6400 - acc: 0.7704 - val_loss: 1.5459 - val_acc: 0.5141
Epoch 5/5
227768/227768 [==============================] - 206s 905us/step - loss: 0.6029 - acc: 0.7843 - val_loss: 1.7544 - val_acc: 0.4960


In [7]:
predicted = model.predict(x_test)
predicted = np.argmax(predicted, axis=1)
print(accuracy_score(y_test_decode, predicted))

0.5277601796924495


In [165]:
a= [0,1,1]
print(len(y_test_decode))
print(np.count_nonzero(y_test_decode))

34726
26884
